Импортируем необходимые библиотеки, прочитаем датасет и разделим его на тренировочный и тестовый.

In [1]:
#импорт библиотек
import numpy as np #для матричных вычислений
import pandas as pd #для анализа и предобработки данных
import matplotlib.pyplot as plt #для визуализации
import seaborn as sns #для визуализации

from sklearn import linear_model #линейные моделиё
from sklearn import tree #деревья решений
from sklearn import ensemble #ансамбли
from sklearn import metrics #метрики
from sklearn import preprocessing #предобработка
from sklearn.model_selection import train_test_split #сплитование выборки
from sklearn.model_selection import GridSearchCV

%matplotlib inline
plt.style.use('seaborn')

C:\Users\tatye\AppData\Local\Temp\ipykernel_5224\586730628.py:16: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn')


In [2]:
data = pd.read_csv('data/molecule.csv')
data.head()

,Activity,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,D1767,D1768,D1769,D1770,D1771,D1772,D1773,D1774,D1775,D1776
0,1,0.000000,0.497009,0.10,0.0,0.132956,0.678031,0.273166,0.585445,0.743663,...,0,0,0,0,0,0,0,0,0,0
1,1,0.366667,0.606291,0.05,0.0,0.111209,0.803455,0.106105,0.411754,0.836582,...,1,1,1,1,0,1,0,0,1,0
2,1,0.033300,0.480124,0.00,0.0,0.209791,0.610350,0.356453,0.517720,0.679051,...,0,0,0,0,0,0,0,0,0,0
3,1,0.000000,0.538825,0.00,0.5,0.196344,0.724230,0.235606,0.288764,0.805110,...,0,0,0,0,0,0,0,0,0,0
4,0,0.100000,0.517794,0.00,0.0,0.494734,0.781422,0.154361,0.303809,0.812646,...,0,0,0,0,0,0,0,0,0,0


In [3]:
#Создаем матрицу наблюдений X и вектор ответов y
X = data.drop(['Activity'], axis=1)
y = data['Activity']

#Разделяем выборку на тренировочную и тестовую в соотношении 80/20. 
#Для сохранения соотношений целевого признака используем параметр stratify (стратифицированное разбиение)
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state = 1, test_size = 0.2)

# <center> Настройки по умолчанию

### <center> Логистическая регрессия 
Построим модель логистической регрессии с параметрами по умолчанию. Зафиксируем метрики.

In [4]:
#Создаем объект класса логистическая регрессия
log_reg = linear_model.LogisticRegression(max_iter = 1000)
#Обучаем модель, минимизируя logloss
log_reg.fit(X_train, y_train)

#предскажем значения на тренировочной выборке и выведем f1 score 
y_train_pred = log_reg.predict(X_train)
print('f1_score на тренировочном наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
#предскажем значения на тестовой выборке и выведем f1 score 
y_test_pred = log_reg.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на тренировочном наборе: 0.89
f1_score на тестовом наборе: 0.78


### <center> Случайный лес
Построим модель случайного леса с параметрами по умолчанию. Зафиксируем метрики.


In [5]:
#Создаем объект класса случайный лес
rf = ensemble.RandomForestClassifier(random_state=42)

#Обучаем модель
rf.fit(X_train, y_train)

#Выводим значения метрики 
y_train_pred = rf.predict(X_train)
print('f1_score на тренировочном наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
y_test_pred = rf.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на тренировочном наборе: 1.00
f1_score на тестовом наборе: 0.81


# <center> Оптимизация и поиск гиперпараметров

### <center> GridSearchCV

Сначала проведем оптимизацию гиперпараметров через поиск по сетке с кросс-валидацией для алгоритма логистической регрессии.

In [6]:
# укажем искомые гиперпараметры:
param_grid = {'penalty': ['l2', 'none'] , # тип регуляризации
              'solver': ['lbfgs', 'sag'], # алгоритм оптимизации
               'C': [0.01, 0.1, 0.3, 0.5, 0.7, 0.9, 1]}, # уровень силы регурялизации
              
# Создаем объект grid_search, передаем в него модель (LogisticRegression), сетку param_grid, число фолдов,
# и n_jobs = -1, чтобы использовать все доступные ядра для расчетов.
grid_search = GridSearchCV(
    estimator=linear_model.LogisticRegression(random_state=1, max_iter=1000), 
    param_grid=param_grid, 
    cv=5, 
    n_jobs = -1
)  
# запускаем кросс-валидацию на тренировочной выборке для каждой комбинации гиперпараметров, ищем лучшие и обучаем модель
grid_search.fit(X_train, y_train) 

# выведем наилучшие значения гиперпараметров
print("Наилучшие значения гиперпараметров: {}".format(grid_search.best_params_))
# строим предсказания и выводим значения метрики f1 на тестовой и тренировочной выборках
y_train_pred = grid_search.predict(X_train)
print('f1_score на тренировочном наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
y_test_pred = grid_search.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))


Наилучшие значения гиперпараметров: {'C': 0.1, 'penalty': 'l2', 'solver': 'sag'}
f1_score на тренировочном наборе: 0.85
f1_score на тестовом наборе: 0.79


Теперь проведем оптимизацию гиперпараметров через поиск по сетке с кросс-валидацией для алгоритма случайного леса.

In [7]:
# укажем искомые гиперпараметры:
param_grid = {'n_estimators': list(range(80, 200, 30)),
              'min_samples_leaf': [1,5,8],
              'max_depth': list(np.linspace(5, 20, 10, dtype=int))}
              
# Создаем объект grid_search, передаем в него модель (Случайный лес), сетку param_grid, число фолдов,
# и n_jobs = -1, чтобы использовать все доступные ядра для расчетов.
grid_search_tree = GridSearchCV(
    estimator=ensemble.RandomForestClassifier(random_state=42), 
    param_grid=param_grid, 
    cv=5, 
    n_jobs = -1
)  
# запускаем кросс-валидацию на тренировочной выборке для каждой комбинации гиперпараметров, ищем лучшие и обучаем модель
grid_search.fit(X_train, y_train) 

# выведем наилучшие значения гиперпараметров
print("Наилучшие значения гиперпараметров: {}".format(grid_search.best_params_))
# строим предсказания и выводим значения метрики f1 на тестовой и тренировочной выборках
y_train_pred = grid_search.predict(X_train)
print('f1_score на тренировочном наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
y_test_pred = grid_search.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))


Наилучшие значения гиперпараметров: {'C': 0.1, 'penalty': 'l2', 'solver': 'sag'}
f1_score на тренировочном наборе: 0.85
f1_score на тестовом наборе: 0.79


### <center> **RandomizedSearchCV**

Теперь переберем и попробуем найти оптимальные гиперпараметры через рандомизированный поиск с использованием класса RandomizedSearchCV.
Сначала для алгоритма логистической регрессии.

In [10]:
# импортируем библиотеку
from sklearn.model_selection import RandomizedSearchCV

# передаем словари с наборами гиперпараметров:
param_distributions = {'penalty': ['l2', 'none'] , # тип регуляризации
              'solver': ['lbfgs', 'sag'], # алгоритм оптимизации
               'C': [0.01, 0.1, 0.3, 0.5, 0.7, 0.9, 1]}, # уровень силы регурялизации
              
      
random_search = RandomizedSearchCV(
    estimator=linear_model.LogisticRegression(random_state=42, max_iter=1000), 
    param_distributions=param_distributions, 
    cv=5, 
    n_iter = 10, 
    n_jobs = -1
)  

# запускаем кросс-валидацию на тренировочной выборке для каждой комбинации гиперпараметров, ищем лучшие и обучаем модель
random_search.fit(X_train, y_train) 

# выведем наилучшие значения гиперпараметров
print("Наилучшие значения гиперпараметров: {}".format(random_search.best_params_))
# строим предсказания и выводим значения метрики f1 на тестовой и тренировочной выборках
y_train_pred = random_search.predict(X_train)
print('f1_score на тренировочном наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
y_test_pred = random_search.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

Наилучшие значения гиперпараметров: {'solver': 'sag', 'penalty': 'l2', 'C': 0.1}
f1_score на тренировочном наборе: 0.85
f1_score на тестовом наборе: 0.79


Теперь проделаем аналогичные действия для алгоритма случайного леса, указав искомые гиперпараметры для этого вида модели. 

In [11]:
# передаем словари с наборами гиперпараметров:
param_grid = {'n_estimators': list(range(100, 200, 20)),
              'min_samples_leaf': [5, 9],
              'max_depth': list(np.linspace(5, 40, 10, dtype=int))
              }
            
random_search_forest = RandomizedSearchCV(
    estimator=ensemble.RandomForestClassifier(random_state=42), 
    param_distributions=param_grid, 
    cv=5,
    n_iter = 15, 
    n_jobs = -1
)   

# запускаем кросс-валидацию на тренировочной выборке для каждой комбинации гиперпараметров, ищем лучшие и обучаем модель
random_search.fit(X_train, y_train) 

# выведем наилучшие значения гиперпараметров
print("Наилучшие значения гиперпараметров: {}".format(random_search.best_params_))
# строим предсказания и выводим значения метрики f1 на тестовой и тренировочной выборках
y_train_pred = random_search.predict(X_train)
print('f1_score на тренировочном наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
y_test_pred = random_search.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

Наилучшие значения гиперпараметров: {'solver': 'sag', 'penalty': 'l2', 'C': 0.1}
f1_score на тренировочном наборе: 0.85
f1_score на тестовом наборе: 0.79


Метрика упорно показывает одно и тоже значение на разных алгоритмах моделей, построенных с использованием поиска по сетке и рандомизированного поиска по сетке. Перейдем к продвинутам способам оптимизации. 

### <center> Hyperopt

Проведем оптимизацию гиперпараметров с помощью алгоритма TPE библиотекой Hyperopt для модели случайного леса.

In [12]:
#делаем импорт библиотеки
from sklearn.model_selection import cross_val_score
import hyperopt
from hyperopt import hp, fmin, tpe, Trials
from click import option
from requests import options


In [13]:
trials = Trials() # используется для логирования результатов

# зададим пространство поиска гиперпараметров
space={'n_estimators': hp.quniform('n_estimators', 50, 300, 20),
       'max_depth' : hp.quniform('max_depth', 15, 40, 1),
       'min_samples_leaf': hp.quniform('min_samples_leaf', 3, 10, 1)
      }

# зафксируем random_state
random_state =42

# создадим функцию для минимизицации метрики
def hyperopt_gb(params, cv=5, X=X_train, y=y_train, random_state=random_state):
    # функция получает комбинацию гиперпараметров в "params"
    params = {'n_estimators': int(params['n_estimators']), 
              'max_depth': int(params['max_depth']), 
              'min_samples_leaf': int(params['min_samples_leaf'])
              }
    # используем эту комбинацию для построения модели
    model = ensemble.RandomForestClassifier(**params, random_state=random_state)
 
    # обучаем модель
    model.fit(X, y)
    score = metrics.f1_score(y, model.predict(X))
    # метрику необходимо минимизировать, поэтому ставим знак минус
    return -score
 
    # начинаем подбор гиперпараметров
best=fmin(hyperopt_gb, # наша функция 
          space=space, # пространство гиперпараметров
          algo=tpe.suggest, # алгоритм оптимизации, установлен по умолчанию, задавать необязательно
          max_evals=25, # максимальное количество итераций
          trials=trials, # логирование результатов
          rstate=np.random.default_rng(random_state), # фиксируем для повторяемости результата
        )
         
# строим и учим модель
model = ensemble.RandomForestClassifier(random_state=random_state, 
        n_estimators=int(best['n_estimators']),  
        max_depth=int(best['max_depth']),
        min_samples_leaf=int(best['min_samples_leaf']))
model.fit(X_train, y_train)

# строим предсказания и выводим значения метрики f1 на тестовой и тренировочной выборках
y_train_pred = model.predict(X_train)
print('f1_score на тренировочном наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
y_test_pred = model.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))


  0%|          | 0/25 [00:00<?, ?trial/s, best loss=?]

100%|██████████| 25/25 [00:52<00:00,  2.09s/trial, best loss: -0.9785801713586292]
f1_score на тренировочном наборе: 0.98
f1_score на тестовом наборе: 0.82


Проведем оптимизацию гиперпараметров с помощью алгоритма TPE библиотекой Hyperopt для модели логистической регрессии.

In [14]:
# зададим пространство поиска гиперпараметров
space = {'penalty': hp.choice(label='penalty', options=['l2', 'none']) , 
        'solver': hp.choice(label='solver', options=['lbfgs', 'sag', 'newton-cg']), 
        'C': hp.uniform('C', 0.01, 1) 
        }

# зафиксируем random_state
random_state = 42

# cоздадим функцию, обучающую модель логистической регрессии по переданным параметрам
def hyperopt(space, cv=5, X=X, y=y, random_state=random_state):
    try:
        model=linear_model.LogisticRegression(
        penalty = space['penalty'],
        solver = space['solver'],
        C=np.abs(float(space['C'])),
        l1_ratio=float(space['l1_ratio']),
        random_state=random_state,
        max_iter=50        
    )        
    except KeyError:
          
        model=linear_model.LogisticRegression(
        penalty = space['penalty'],
        solver = space['solver'],
        C=np.abs(float(space['C'])),
        random_state=random_state,
        max_iter=50
           )
    score = cross_val_score(model, X, y, cv=cv, scoring="f1", n_jobs=-1).mean()   # применим  cross validation 
    return -score # метрику необходимо минимизировать, поэтому ставим знак минус

    # начинаем подбор гиперпараметров
trials = Trials() # используется для логирования результатов  

best=fmin(hyperopt, # наша функция 
          space=space, # пространство гиперпараметров
          algo=tpe.suggest, # алгоритм оптимизации, установлен по умолчанию, задавать необязательно
          max_evals=30, # максимальное количество итераций
          trials=trials, # логирование результатов
          rstate=np.random.default_rng(random_state)# фиксируем для повторяемости результата
         )

100%|██████████| 30/30 [03:19<00:00,  6.65s/trial, best loss: -0.7916411179230474]


In [16]:
from hyperopt import space_eval
# строим и учим модель
model_log = linear_model.LogisticRegression(
        penalty=space_eval(space, best)['penalty'],
        solver=space_eval(space, best)['solver'],
        C=float(best['C']),
        random_state=random_state,
        max_iter=1000)
model_log.fit(X_train, y_train)

# строим предсказания и выводим значения метрики f1 на тестовой и тренировочной выборках
y_train_pred = model_log.predict(X_train)
print('f1_score на тренировочном наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
y_test_pred = model_log.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на тренировочном наборе: 0.83
f1_score на тестовом наборе: 0.80


### <center> Optuna

Проведем подборку гиперпараметров методом Optuna для логистической регрессии.

In [17]:
# Устанавливаем библиотеку
import optuna

In [18]:
random_state = 42

# создадим функцию, обучающую модель LogisticRegression по переданным гиперпараметрам
def optuna_log(trial):
    # задаем пространства поиска гиперпараметров
    penalty = trial.suggest_categorical('penalty', ['l2', 'none'])
    solver = trial.suggest_categorical('solver', ['lbfgs', 'sag', 'newton-cg'])
    C = trial.suggest_float('C', 0.1, 1)  

   # создаем модель
    model=linear_model.LogisticRegression(
        penalty = penalty,
        solver = solver,
        C=C, 
        random_state=random_state,
        max_iter=50        
    )   
  # обучаем модель
    score = cross_val_score(model, X, y, cv=5, scoring="f1", n_jobs=-1).mean()

    return score

In [20]:
# cоздаем объект исследования для нашего набора гиперпараметров
# можем напрямую указать, что нам необходимо максимизировать метрику direction="maximize"
study = optuna.create_study(study_name="LogisticRegression", direction="maximize")
# ищем лучшую комбинацию гиперпараметров n_trials=15 раз
study.optimize(optuna_log, n_trials=15)

# создадим и обучим модель
model_optuna_log = linear_model.LogisticRegression(**study.best_params,random_state=random_state)
model_optuna_log.fit(X_train, y_train)

[I 2023-07-07 18:43:33,712] A new study created in memory with name: LogisticRegression
[I 2023-07-07 18:43:36,546] Trial 0 finished with value: 0.7757525816757611 and parameters: {'penalty': 'none', 'solver': 'sag', 'C': 0.12908485596740035}. Best is trial 0 with value: 0.7757525816757611.
[I 2023-07-07 18:43:40,854] Trial 1 finished with value: 0.7778538666768492 and parameters: {'penalty': 'l2', 'solver': 'newton-cg', 'C': 0.3860772996762678}. Best is trial 1 with value: 0.7778538666768492.
[I 2023-07-07 18:43:43,621] Trial 2 finished with value: 0.7757525816757611 and parameters: {'penalty': 'none', 'solver': 'sag', 'C': 0.8455282098880559}. Best is trial 1 with value: 0.7778538666768492.
[I 2023-07-07 18:43:46,618] Trial 3 finished with value: 0.7757525816757611 and parameters: {'penalty': 'none', 'solver': 'sag', 'C': 0.2108414055395334}. Best is trial 1 with value: 0.7778538666768492.
[I 2023-07-07 18:43:49,590] Trial 4 finished with value: 0.7793857907075743 and parameters: {'p

LogisticRegression(C=0.12972084632917036, random_state=42, solver='newton-cg')

In [21]:
# строим предсказания и выводим значения метрики f1 на тестовой и тренировочной выборках
y_train_pred = model_optuna_log.predict(X_train)
print('f1_score на тренировочном наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
y_test_pred = model_optuna_log.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на тренировочном наборе: 0.85
f1_score на тестовом наборе: 0.79


И теперь проведем подборку гиперпараметров методом Optuna для случайного леса.

In [24]:
def optuna_rf(trial):
  # задаем пространства поиска гиперпараметров
  n_estimators = trial.suggest_int('n_estimators', 100, 295, 15)
  max_depth = trial.suggest_int('max_depth', 7, 29, 2)
  min_samples_leaf = trial.suggest_int('min_samples_leaf', 3, 7, 1)

  # создаем модель
  model = ensemble.RandomForestClassifier(n_estimators=n_estimators,
                                          max_depth=max_depth,
                                          min_samples_leaf=min_samples_leaf,
                                          random_state=42)
  # обучаем модель
  model.fit(X_train, y_train)
  score = metrics.f1_score(y_train, model.predict(X_train))

  return score

# cоздаем объект исследования
study = optuna.create_study(study_name="RandomForestClassifier", direction="maximize")
# ищем лучшую комбинацию гиперпараметров n_trials раз
study.optimize(optuna_rf, n_trials=25)

[I 2023-07-07 18:48:52,102] A new study created in memory with name: RandomForestClassifier


[I 2023-07-07 18:48:54,074] Trial 0 finished with value: 0.9387008234217749 and parameters: {'n_estimators': 175, 'max_depth': 11, 'min_samples_leaf': 3}. Best is trial 0 with value: 0.9387008234217749.
[I 2023-07-07 18:48:57,484] Trial 1 finished with value: 0.9336594313665546 and parameters: {'n_estimators': 295, 'max_depth': 27, 'min_samples_leaf': 6}. Best is trial 0 with value: 0.9387008234217749.
[I 2023-07-07 18:48:59,606] Trial 2 finished with value: 0.9272616136919315 and parameters: {'n_estimators': 190, 'max_depth': 29, 'min_samples_leaf': 7}. Best is trial 0 with value: 0.9387008234217749.
[I 2023-07-07 18:49:02,448] Trial 3 finished with value: 0.9578754578754579 and parameters: {'n_estimators': 235, 'max_depth': 13, 'min_samples_leaf': 3}. Best is trial 3 with value: 0.9578754578754579.
[I 2023-07-07 18:49:03,649] Trial 4 finished with value: 0.9332516840171464 and parameters: {'n_estimators': 100, 'max_depth': 23, 'min_samples_leaf': 6}. Best is trial 3 with value: 0.957

In [25]:
# создадим и обучим модель
model_optuna_rf = ensemble.RandomForestClassifier(**study.best_params,random_state=random_state, )
model_optuna_rf.fit(X_train, y_train)

# строим предсказания и выводим значения метрики f1 на тестовой и тренировочной выборках
y_train_pred = model_optuna_log.predict(X_train)
print('f1_score на тренировочном наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
y_test_pred = model_optuna_log.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на тренировочном наборе: 0.85
f1_score на тестовом наборе: 0.79


## <center> Вывод

Мы провели оптимизацию гиперпараметров с помощью разных алгоритмов на модели случайного леса и логистической регрессии. 
Выведем все значения f1 метрики на тестовой выборке в таблицу. 

<img src = 'data/2.png'> 


Для логистической регрессии все способы оптимизации дали результаты лучше, чем базовый с настройками по умолчанию, самый лучший результат метрики дал Hyperopt.

Для модели случайного леса на переданных комбинациях гиперпараметров метрика f1 оказалось лучшей так же у Hyperopt. Остальные способы оптимизации показали метрку чуть ниже, чем baseline вариант, созданный с настройками по умолчанию. 

